## 1. IMPORTS

In [ ]:
import os
print(f"# --- VERIFICATION DU REPERTOIRE COURANT : {os.getcwd()}")
import sys
# --- ACCES A SRC
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")

# ---AJOUT DE src AU sys.path
if src_path not in sys.path:
    sys.path.append(src_path)
print(f"# --- CHEMIN AJOUTE AU sys.path : {src_path}")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer

## 2. CHARGEMENT DES DONNEES

In [ ]:
# --------------------------------------
# --- CHARGEMENT DES DONNEES PRINCIPALES
# --------------------------------------
full_explo_wo = pd.read_csv("../data/processed/full_explo_wo_wet.csv")

print(f"""# --- DONNEES NETTOYEES SANS LES TAGS ENCODES : 
          # --- DIMENSIONS : {full_explo_wo.shape}
          # --- COLONNES : {full_explo_wo.columns.tolist()}""")
# --------------------------------------------------------------------------------
# CHARGEMENT DES TAGS ENCODES : A DECOMMENTER SSI BESOIN
# --------------------------------------------------------------------------------
# tags_df_full = pd.read_csv("../data/processed/tags_df_full.csv")
# --- AVEC AU BESOIN UNE CONCATENATION AVEC LES DONNEES PRINCIPALES
# full_explo_wo = pd.concat([full_explo_wo, tags_df_full[selected_tags]], axis=1)
# print(f"""# --- TAGS ENCODES : 
#           # --- DIMENSIONS : {tags_df_full.shape}
#           # --- COLONNES : {tags_df_full.columns.tolist()}""")
# --------------------------------------------------------------------------------

## 3. DECOUPAGE TEMPOREL DES DONNEES

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# --- ON CONVERTIT AU FORMAT DATETIME POUR EXPLOITER LA COLONNE QUI PERMETTRA LE DECOUPAGE TEMPOREL DES DONNEES
# --------------------------------------------------------------------------------------------------------------
full_explo_wo["CreationDate"] = pd.to_datetime(full_explo_wo["CreationDate"])
# ----------------------------------------------------------------------------------------------
# --- ON CREE LA COLONNE MOIS POUR IDENTIFIER LE MOIS DE LA DATE DE CREATION DE CHAQUE QUESTION
# ----------------------------------------------------------------------------------------------
# --- FORMAT  yyyy-mm
full_explo_wo["CreationMonth"] = full_explo_wo["CreationDate"].dt.to_period("M")
# --------------------------------
# ON REGROUPE LES CORPUS PAR MOIS
# --------------------------------
# --- LE CORPUS CHOISI EST SPECIFIQUE AU MODELE QUI SERA APPLIQUE POUR L'ANALYSE DRIFT (MODELE SBERT)
monthly_texts = {
    str(month): group["clean_title_body_embed"].tolist()
    for month, group in full_explo_wo.groupby("CreationMonth")
}
print(f"# --- MOIS DISPONIBLES A L'ANALYSE : \n{list(monthly_texts.keys())}")
# ----------------------------------------
# --- CREATION D'UN DICT DE DATAFRAMES MENSUELS
# ----------------------------------------
monthly_dfs = {
    str(month): group.reset_index(drop=True)
    for month, group in full_explo_wo.groupby("CreationMonth")
}
print(f"# --- LES TAGS DES QUESTIONS DE JANVIER 2015 SONT : \n{monthly_dfs["2015-01"]["Tags"]}")

## 4. SELECTION TEMPORELLE DES DONNEES

In [ ]:
# -------------------------------------------------------------
# FILTRAGE DES DONNEES : ON NE GARDE QU'UNE PERIODE DE 12 MOIS
# -------------------------------------------------------------
# --- SELECTION DES MOIS DE JANVIER A DECEMBRE 2014
selected_months = [
    '2014-01', '2014-02', '2014-03', '2014-04',
    '2014-05', '2014-06', '2014-07', '2014-08',
    '2014-09', '2014-10', '2014-11', '2014-12'
]
# --- FILTRAGE DES DONNEES
monthly_dfs_filtered = {
    month: monthly_dfs[month]
    for month in selected_months
    if month in monthly_dfs
}
# --- VERIFICATION DU FILTRAGE
print("# --- APRES FILTRAGE, DONNEES DISPONIBLES POUR CETTE PERIODE :")
for month, df in monthly_dfs_filtered.items():
    print(f"- {month} : {len(df)} textes")
    print(df.columns)
# -------------------------------
# --- ANALYSE DESCRIPTIVE RAPIDE
# -------------------------------
# --- CREATION DATAFRAME AVEC VOLUME TEXTUEL POUR CHAQUE MOIS DE DONNEES
volume_df = pd.DataFrame({
    'mois': list(monthly_dfs_filtered.keys()),
    'nb_textes': [len(df) for df in monthly_dfs_filtered.values()]
})
# --- STATISTIQUES DESCRIPTIVES
print("# --- STATISTIQUES SUR LA DISPERSION DU NOMBRE DE TEXTES POUR CHAQUE MOIS :")
print(volume_df['nb_textes'].describe())
# -------------------------------------------------------------------------------------
# --- VISUALISATION DE LA DISTRIBUTION DE TEXTE SUR LES MOIS DE LA PERODE SELECTIONNEE
# -------------------------------------------------------------------------------------
# --- CREATION BARPLOT
plt.figure(figsize=(10, 6))
sns.barplot(x='mois', y='nb_textes', data=volume_df, hue='mois', palette='viridis', dodge=False, legend=False)
# --- QUELQUES CHIFFRES POUR AFFICHER DES REPERES VISUELS
mean_volume = volume_df['nb_textes'].mean()
median_volume = volume_df['nb_textes'].median()
min_volume = volume_df['nb_textes'].min()
max_volume = volume_df['nb_textes'].max()
plt.axhline(mean_volume, color='violet', linestyle='--', linewidth=1, label=f'Moyenne ({mean_volume:.1f})')
plt.axhline(median_volume, color='darkblue', linestyle='--', linewidth=1, label=f'Médiane ({median_volume:.1f})')
plt.axhline(min_volume, color='green', linestyle='--', linewidth=1, label=f'Minimum ({min_volume:.1f})')
plt.axhline(max_volume, color='orange', linestyle='--', linewidth=1, label=f'Maximum ({max_volume:.1f})')
plt.title("NOMBRE DE TEXTES PAR MOIS (2014)", fontsize=14)
plt.xlabel("MOIS")
plt.ylabel("NOMBRE DE TEXTES")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

## 5. ENCODAGE SBERT DES CORPUS MENSUELS

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES D' EXPLORATION
# -------------------------------------------
import importlib
import tags_suggester.eda.eda_analysis as eda
importlib.reload(eda)

# --------------------------------------------------------------------------------
# CHARGEMENT DU MODELE SBERT SAUVEGARDE LORS DE LA PHASE EXPLORATOIRE DU PROJET
# --------------------------------------------------------------------------------
model = SentenceTransformer("models/sbert/sbert_model")
# --------------------------------------------
# --- ENCODAGE SBERT DES TEXTES DE CHAQUE MOIS
# --------------------------------------------
monthly_embeddings = {}
print("\n# --- ENCODAGE SBERT EN COURS ...")
for month, df in monthly_dfs_filtered.items():
    texts = df["clean_title_body_embed"].fillna("").tolist()
    print(f"# --- POUR LE MOIS DE {month} : {len(texts)} TEXTES A ENCODER ---")
    embeddings = eda.encode_sbert_corpus(texts, model=model, batch_size=32)
    monthly_embeddings[month] = embeddings
    print(f"# --- ENCODAGE EFFECTUE POUR LE MOIS DE {month} → shape : {embeddings.shape} ---")

print("\n# --- TOUS LES MOIS ONT BIEN ETE ENCODES AVEC SUCCES.")

## 6. ANALYSE DE DRIFT SUPERVISEE - EVIDENTLY AI

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES D'EXPLORATION
# -------------------------------------------
import importlib
import tags_suggester.eda.eda_analysis as eda
importlib.reload(eda)
import tags_suggester.modeling.modeling as mdl
importlib.reload(mdl)
from sklearn.exceptions import UndefinedMetricWarning
import warnings
from sklearn.exceptions import InconsistentVersionWarning
warnings.filterwarnings("ignore", category=InconsistentVersionWarning)
import os
output_dir = "evidently_reports"
os.makedirs(output_dir, exist_ok=True)

# --- IMPORTS POUR EVIDENTLY 0.5.1 ---
from evidently.report import Report
from evidently.metric_preset import ClassificationPreset
from evidently import ColumnMapping

# -----------------------------------------------------------------------------
# --- CHARGEMENT DU BINARIZER POUR ENCODER LES TAGS A CHAQUE DATAFRAME MENSUEL
# ------------------------------------------------------------------------------
import joblib
mlb = joblib.load("models/tags/multilabel_binarizer_full.pkl")
print("# --- BINARIZER CHARGÉ AVEC SUCCÈS.")
print(f"NOMBRE DE TAGS : {len(mlb.classes_)}")
print("EXEMPLES DE TAGS :", mlb.classes_[:10])

# ---------------------------------------------------------------------------------------------------
# --- CHARGEMENT DE L'INSTANCE DU MODÈLE DE RÉGRESSION LOGISTIQUE FONCTIONNANT AVEC DES INPUTS SBERT
# ---------------------------------------------------------------------------------------------------
log_reg_model = joblib.load("models/logreg/logreg_sbert.joblib")
print("# --- MODÈLE DE RÉGRESSION LOGISTIQUE SBERT CHARGÉ :")
print("# --- TYPE DU MODÈLE :", type(log_reg_model))
print("# --- ÉTAPES DU PIPELINE :", log_reg_model.named_steps)

# -----------------------------------------------------------------
# --- ÉVALUATION DU MODÈLE DE PRÉDICTION POUR CHAQUE MOIS DE DONNÉES
# -----------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

results = []
monthly_dashboards = {}

for month in monthly_dfs_filtered:
    df = monthly_dfs_filtered[month]
    embeddings = monthly_embeddings[month]
    
    # --- Évaluation classique
    scores = mdl.evaluate_month(
        month=month,
        df=df,
        embeddings=embeddings,
        model=log_reg_model,
        mlb=mlb
    )
    scores["mois"] = month  #  utile pour le tri
    results.append(scores)
    
    # --- Ajout des colonnes y_true et y_pred pour Evidently
    y_true = mlb.transform(df["Tags"])
    y_pred = log_reg_model.predict(embeddings)
    
    df["target"] = [mlb.classes_[row.argmax()] if row.sum() > 0 else "none" for row in y_true]
    df["prediction"] = [mlb.classes_[row.argmax()] if row.sum() > 0 else "none" for row in y_pred]
    # TODO : numerical_features et categorical_features : on met rien?
    column_mapping = ColumnMapping(
        target="target",
        prediction="prediction",
        numerical_features=["Score", "ViewCount", "AnswerCount", "TagCount",
        "length_words_raw", "length_words_clean", "reduction_ratio_global", ],  # à compléter si besoin
        categorical_features=[]
    )

    # --- Création du rapport Evidently
    report = Report(metrics=[ClassificationPreset()])
    # TODO : df ne peut servir à la fois de référence et de comparé
    report.run(reference_data=df, current_data=df, column_mapping=column_mapping)

    monthly_dashboards[month] = report
    
    # report.save_html(f"drift_report_{month}.html")
    report.save_html(os.path.join(output_dir, f"drift_report_{month}.html"))

# --- INCLURE SYNTHESE COMPARATIVE DES 12 MOIS
from evidently.report import Report
from evidently.metric_preset import ClassificationPreset

# Comparaison entre janvier et décembre
report_global = Report(metrics=[ClassificationPreset()])
report_global.run(
    reference_data=monthly_dfs_filtered["2014-01"],
    current_data=monthly_dfs_filtered["2014-12"],
    column_mapping=column_mapping
)

# Sauvegarde du rapport en HTML
report_global.save_html(os.path.join(output_dir, "report_global_janv_vs_dec.html"))


# Résultats sous forme de DataFrame
results_df = pd.DataFrame(results).sort_values("mois")
display(results_df)

# Exemple d'affichage et export
monthly_dashboards["2014-02"].show()
# monthly_dashboards["2015-02"].save("drift_report_2015_02.html")


In [ ]:
from evidently.report import Report
from evidently.metric_preset import ClassificationPreset
from evidently import ColumnMapping



In [ ]:
import evidently
print(evidently.__version__)


## 7. ANALYSE DE DRIFT NON SUPERVISEE

### 7.1. DISTANCE ENTRE CENTROIDS

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
import pandas as pd

# Choix du mois de référence
reference_month = '2014-01'
reference_centroid = monthly_embeddings[reference_month].mean(axis=0)

# Calcul des distances cosinus
drift_scores_cosine = {}

for month, emb in monthly_embeddings.items():
    centroid = emb.mean(axis=0)
    distance = cosine_distances([reference_centroid], [centroid])[0][0]
    drift_scores_cosine[month] = distance

# Résultat sous forme de DataFrame
df_drift_cosine = pd.DataFrame.from_dict(drift_scores_cosine, orient='index', columns=['cosine_distance'])
df_drift_cosine.sort_index(inplace=True)
display(df_drift_cosine)
df_drift_cosine.plot(title="Drift non supervisé - Distance Cosine", figsize=(10, 5))


### 7.2. PCA - VISUALISATION DES EMBEDDINGS

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Réduction à 2 dimensions
pca = PCA(n_components=2)

plt.figure(figsize=(12, 6))
for month, emb in monthly_embeddings.items():
    reduced = pca.fit_transform(emb)
    plt.scatter(reduced[:, 0], reduced[:, 1], label=month, alpha=0.5)

plt.title("Projection PCA des embeddings mensuels")
plt.legend()
plt.grid(True)
plt.show()


### 7.3. TESTS STATISTIQUES - WASSERSTEIN - KOLMOGOROV SMIRNOV

In [ ]:
from scipy.stats import wasserstein_distance, ks_2samp

# On compare chaque mois au mois de référence
drift_stats = {}

for month, emb in monthly_embeddings.items():
    if month == reference_month:
        continue

    # Flatten pour comparaison 1D (approche simplifiée)
    ref_flat = monthly_embeddings[reference_month].flatten()
    month_flat = emb.flatten()

    w_dist = wasserstein_distance(ref_flat, month_flat)
    ks_stat, ks_pval = ks_2samp(ref_flat, month_flat)

    drift_stats[month] = {
        'wasserstein': w_dist,
        'ks_stat': ks_stat,
        'ks_pval': ks_pval
    }

# Résultat en DataFrame
df_drift_stats = pd.DataFrame.from_dict(drift_stats, orient='index')
df_drift_stats.sort_index(inplace=True)
df_drift_stats.plot(subplots=True, figsize=(12, 8), title="Drift statistique par rapport à 2023-01")


### 7.4. AVEC EVIDENTLY AI

In [ ]:
import evidently
print("# --- Version Evidently :", evidently.__version__)


from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Comparaison entre le mois de référence et un autre mois
ref_df = monthly_dfs_filtered['2014-01']
target_df = monthly_dfs_filtered['2014-02']

report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=ref_df, current_data=target_df)
report.show()
